In [17]:
import pandas as pd
from pathlib import Path


file_path = "ProMOB_2022.xlsb"

try:
    # Read all sheet names from the XLSB file
    excel_file = pd.ExcelFile(file_path, engine='pyxlsb')
    sheet_names = excel_file.sheet_names
    
    print("Available sheets:")
    for sheet in sheet_names:
        print(f"- {sheet}")
    
    sheets = sheet_names

except FileNotFoundError:
    print(f"Error: File '{file_path}' not found")
    sheets = None
except Exception as e:
    print(f"Error: {str(e)}")
    sheets = None



Available sheets:
- Evolução
- Técnico
- Produtor
- Ranking
- Ranking Semanal
- Plano de Ação
- Geral
- Check Duplicidade
- Impressão 09-08-21 a 09-08-22
- Impressão após 10-08-22
- Lançamento
- Ponte
- Dados
- REAVALIAÇÃO
- BLOQUEADOS
- Próximo Check-list
- Último Check-list
- Consultas


In [18]:
if sheets and 'Dados' in sheets:
    try:
        df_resultado_ProMOB = excel_file.parse('Dados')
        print(df_resultado_ProMOB.head())
    except Exception as e:
        print(f"Error reading sheet 'Dados': {str(e)}")
else:
    print("Sheet 'Dados' not found or sheets not loaded.")

    Data  Aviário  Lote Avaliador       Digitador    1    2    3  4    5  ...  \
0  44564     1180     6   ISABELA  ANA CLAUDIA W.    C    C    C  C    C  ...   
1  44564      166   152    MAYARA  ANA CLAUDIA W.    C    C  N/C  C    C  ...   
2  44564     1092    17    MAYARA  ANA CLAUDIA W.    C    C    C  C  N/C  ...   
3  44564     1219     3   ISABELA  ANA CLAUDIA W.    C    C    C  C    C  ...   
4  44564      115   151   ISABELA  ANA CLAUDIA W.  N/C  N/C  NaN  C    C  ...   

    48   49      Nota Núcleo              Proprietário  \
0  NaN  NaN  0.950000    362    MARILENE BURIN GLAESER   
1  NaN  NaN  0.969697      9      EDMIR ANTONIO SOARES   
2  NaN  NaN  0.910000    353  ADEMIR FERREIRA DE PADUA   
3  NaN  NaN  0.939394    376          GABRIEL CANEVESE   
4  NaN  NaN  0.845361    123           VALDIR CANEVESE   

                              Técnico     Mês Aviário-Lote Semana  \
0  INAIARA CALCAGNO RODRIGUES BUTTINI  202201       1180-6      2   
1           LEONARDO HENRI

In [19]:
# Select specific columns
df_resultado_ProMOB = df_resultado_ProMOB[['Aviário-Lote','Núcleo','Mês','Técnico','Nota']]

# Remove duplicates based on 'Aviário-Lote'
df_resultado_ProMOB = df_resultado_ProMOB.drop_duplicates(subset=['Aviário-Lote'])

# Extract first 4 characters from 'Mês' column and rename to 'Ano'
df_resultado_ProMOB['Ano'] = df_resultado_ProMOB['Mês'].astype(str).str[:4]
df_resultado_ProMOB = df_resultado_ProMOB.drop('Mês', axis=1)

# Extract only the first name from 'Técnico' column
df_resultado_ProMOB['Técnico'] = df_resultado_ProMOB['Técnico'].str.split().str[0]

# Multiply the 'Nota' column by 100
df_resultado_ProMOB['Nota'] = df_resultado_ProMOB['Nota'] * 100
# Round the 'Nota' column to 1 decimal place
df_resultado_ProMOB['Nota'] = df_resultado_ProMOB['Nota'].round(1)

print(df_resultado_ProMOB.head())


  Aviário-Lote  Núcleo   Técnico  Nota   Ano
0       1180-6     362   INAIARA  95.0  2022
1      166-152       9  LEONARDO  97.0  2022
2      1092-17     353  LEONARDO  91.0  2022
3       1219-3     376   WILLIAN  93.9  2022
4      115-151     123   WILLIAN  84.5  2022


In [20]:
import pandas as pd
import sqlite3
from pathlib import Path

# Define the file path
file_path = "Indicadores Fomento_2025.xlsb"

# Database file name
db_file = "fomento_data.db"

try:
    # Read all sheet names from the XLSB file
    excel_file = pd.ExcelFile(file_path, engine='pyxlsb')
    sheet_names = excel_file.sheet_names
    
    print("Available sheets:")
    for sheet in sheet_names:
        print(f"- {sheet}")
    
    sheets = sheet_names

except FileNotFoundError:
    print(f"Error: File '{file_path}' not found")
    sheets = None
except Exception as e:
    print(f"Error: {str(e)}")
    sheets = None

if sheets and 'BD_Resultado Lotes' in sheets:
    try:
        df_resultado_lotes = excel_file.parse('BD_Resultado Lotes')
        # Select specific columns
        df_resultado_lotes = df_resultado_lotes[['Fazenda', 'Proprietario', 'Conversão Alimentar', 'CLASSIFICAÇÃO']]

        # Connect to SQLite database
        conn = sqlite3.connect(db_file)
        
        # Write the dataframe to a table in the SQLite database
        df_resultado_lotes.to_sql('BD_Resultado_Lotes', conn, if_exists='replace', index=False)
        
        # Commit the changes and close the connection
        conn.commit()
        conn.close()
        
        print(f"Data from sheet 'BD_Resultado Lotes' saved to '{db_file}' successfully.")

    except Exception as e:
        print(f"Error processing sheet 'BD_Resultado Lotes': {str(e)}")
else:
    print("Sheet 'BD_Resultado Lotes' not found or sheets not loaded.")

Error: File 'Indicadores Fomento_2025.xlsb' not found
Sheet 'BD_Resultado Lotes' not found or sheets not loaded.
